In [2]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
import datetime
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from tqdm import tqdm

2024-07-15 22:25:53.096328: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 22:25:53.349082: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 22:25:53.349274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 22:25:53.380408: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-15 22:25:53.452777: I tensorflow/core/platform/cpu_feature_guar

In [8]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import SGD, Adam

In [5]:
import cv2
import urllib
import requests
import PIL.Image

from bs4 import BeautifulSoup

In [6]:
#downloading ship synset
ship_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n03095699")
ship_soup = BeautifulSoup(ship_page.content, "html.parser")

#downloading bike synset:
bike_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n03792782")
bike_soup = BeautifulSoup(bike_page.content, "html.parser")

ship_soup_str = str(ship_soup)
ship_soup_split_url = ship_soup_str.split("\r\n")

bike_soup_str = str(bike_soup)
bike_soup_split_url = bike_soup_str.split("\r\n")

In [ ]:
os.mkdir('./content')
os.mkdir('./content/train')
os.mkdir('./content/train/ships')
os.mkdir('./content/train/bikes')
os.mkdir('./content/validation')
os.mkdir('./content/validation/ships')
os.mkdir('./content/validation/bikes')

In [12]:
print("le: ", len(bike_soup_split_url))

le:  1


In [8]:
image_rows, image_cols = 32, 32

input_shape = (image_rows, image_cols, 3)

def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

n_training_images = 100

#train images for ship
for progress in tqdm(range(n_training_images)):
    if not ship_soup_split_url[progress] == None:
        try:
            I = url_to_image(ship_soup_split_url[progress])
            if len(I.shape) == 3:
                save_path = "./content/train/ships/img" + str(progress) + ".jpeg"
                cv2.imwrite(save_path, I)
        except:
            None

#train images for bikes
for progress in tqdm(range(n_training_images)):
    if not bike_soup_split_url[progress] == None:
        try:
            I = url_to_image(bike_soup_split_url[progress])
            if len(I.shape) == 3:
                save_path = "./content/train/bikes/img" + str(progress) + ".jpeg"
                cv2.imwrite(save_path, I)
        except:
            None

#validation images for ships
for progress in tqdm(range(50)):
    if not ship_soup_split_url[n_training_images+progress] == None:
        try:
            I = url_to_image(ship_soup_split_url[n_training_images+progress])
            if len(I.shape) == 3:
                save_path = "./content/validation/ships/img" + str(progress) + ".jpeg"
                cv2.imwrite(save_path, I)
        except:
            None


#validation images for bikes
for progress in tqdm(range(50)):
    if not bike_soup_split_url[n_training_images+progress] == None:
        try:
            I = url_to_image(bike_soup_split_url[n_training_images+progress])
            if len(I.shape) == 3:
                save_path = "./content/validation/bikes/img" + str(progress) + ".jpeg"
                cv2.imwrite(save_path, I)
        except:
            None

hisfhihihh:  1


  1%|          | 1/100 [00:00<00:00, 7145.32it/s]


IndexError: list index out of range

In [26]:
#defining model

class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(
            96, 
            kernel_size=(11, 11), 
            strides=4, 
            padding="valid", 
            activation="relu", 
            input_shape=input_shape,
            kernel_initializer="he_normal"
        ))
        
        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides = (2, 2),
            padding = "valid",
            data_format = None
        ))

        self.add(Conv2D(
            256,
            kernel_size=(5, 5),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides = (2, 2),
            padding="valid",
            data_format=None
        ))

        self.add(Conv2D(
            384,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(Conv2D(
            384,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides=(2, 2),
            padding="valid",
            data_format=None
        ))

        self.add(Flatten())

        self.add(Dense(4096, activation="relu"))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(1000, activation="relu"))
        self.add(Dense(num_classes, activation="softmax"))

        self.compile(
            optimizer=Adam(0.001),
            loss = 'categorical_crossentropy',
            metrics=["accuracy"]
        )


In [32]:
num_classes = 3

model = AlexNet((227, 227, 3), num_classes)

epochs = 10
batch_size = 32

train_dir = "/home/abhinav/Documents/Work/Hobby/Datasets/Imagenet/synsets"
valid_dir = "/home/abhinav/Documents/Work/Hobby/Datasets/Imagenet/valid_synsets"

Image_height = 227
Image_width = 227

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (Image_height, Image_width),
    color_mode="rgb",
    batch_size=batch_size,
    seed=1,
    shuffle=True,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(Image_height, Image_width),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

train_num = train_generator.samples
valid_num = valid_generator.samples

Found 3605 images belonging to 3 classes.
Found 264 images belonging to 3 classes.


In [28]:
model_dir = "./myalexnetmodel.h5"

In [24]:
os.mkdir("./logs")
os.mkdir("./logs/fit")

FileExistsError: [Errno 17] File exists: './logs'

In [36]:
log_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%m%Y-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks_list = [tensorboard_callback]

model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=train_num//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_num//batch_size,
    callbacks=callbacks_list,
    verbose=1
)

model.summary()

model.save(model_dir)

Epoch 1/10


2024-07-16 03:06:40.100725: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 35.45MiB (rounded to 37171200)requested by op alex_net_5/conv2d_10/Relu
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-07-16 03:06:40.100855: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-07-16 03:06:40.100894: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 120, Chunks in use: 119. 30.0KiB allocated for chunks. 29.8KiB in use in bin. 826B client-requested in use in bin.
2024-07-16 03:06:40.100914: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 10, Chunks in use: 10. 5.0KiB allocated for chunks. 5.0KiB in use in bin. 3.8KiB client-requested in use in bin.
2024-07-

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL ResourceExhaustedError REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/abhinav/.local/lib/python3.10/site-packages/tensorflow/python/framework/errors_impl.py(377): __init__
  /home/abhinav/.local/lib/python3.10/site-packages/tensorflow/python/eager/execute.py(53): quick_execute
  /home/abhinav/.local/lib/python3.10/site-packages/tensorflow/python/eager/context.py(1486): call_function
  /home/abhinav/.local/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(251): call_flat
  /home/abhinav/.local/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(216): call_preflattened
  /home/abhinav/.local/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/concrete_function.py(1323): _call_flat
  /home/abhinav/.local/lib/python3.10/site-packages/tensorflow/python/eager/polym